# Load modul/library dan dataset

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout

# Load dataset CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
# Cek bentuk gambar
print(X_train.shape)
print(X_test.shape)

In [ ]:
# Shape target
print(y_train.shape)
print(y_test.shape)

# Preprocessing gambar

In [ ]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Tampilkan beberapa gambar pertama
fig, axes = plt.subplots(ncols=6, nrows=2, figsize=(12, 4))
index = 0
for i in range(2):
    for j in range(6):
        axes[i,j].set_title(label_names[y_train[index][0]])
        axes[i,j].imshow(X_train[index])
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        index += 1
plt.show()

# Ubah jadi grayscale

In [ ]:
# Convert jadi grayscalse
X_train = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_train])
X_test = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_test])

# Sekarang chanel warnanya sudah hilang
print(X_train.shape)
print(X_test.shape)

In [ ]:
# Tampilkan beberapa gambar pertama
fig, axes = plt.subplots(ncols=6, nrows=2, figsize=(12, 4))
index = 0
for i in range(2):
    for j in range(6):
        axes[i,j].set_title(label_names[y_train[index][0]])
        axes[i,j].imshow(X_train[index])
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        index += 1
plt.show()

# Normalisasi

In [ ]:
print('max:', X_train.max(), ', min:', X_train.min())

# Normalisasi
X_train  = X_train/255
X_test  = X_test/255

print('max:', X_train.max(), ', min:', X_train.min())

# Preprocessing label

In [ ]:
print(y_train[:5])

In [ ]:
# Kita bisa pakai OneHotEncoder dari modul Sklearn
one_hot_encoder = OneHotEncoder(sparse=False)

# Fit data berdasarkan y_train (belum mengubah y_train jadi one-hot)
one_hot_encoder.fit(y_train)

# Sekarang baru y_train dan y_test berubah jadi one-hot
y_train = one_hot_encoder.transform(y_train)
y_test = one_hot_encoder.transform(y_test)

In [ ]:
print(y_train[:5])

# Membuat model CNN

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', 
                 padding='same', input_shape=(32, 32, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', 
                 padding='same'))
model.add(MaxPool2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', 
                 padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', 
                 padding='same'))
model.add(MaxPool2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', 
                 padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', 
                 padding='same'))
model.add(MaxPool2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))

In [ ]:
# compile model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=4, 
                    batch_size=64, validation_data=(X_test, y_test))

# Prediksi & Confusion matrix

In [ ]:
predictions = model.predict(X_test)
print(predictions[0])

In [ ]:
predictions = one_hot_encoder.inverse_transform(predictions)
y_test = one_hot_encoder.inverse_transform(y_test)

In [ ]:
cm = confusion_matrix(y_test, predictions)

plt.figure(figsize=(9,9))
sns.heatmap(cm, cbar=False, xticklabels=label_names, yticklabels=label_names, 
            fmt='d', annot=True, cmap=plt.cm.Blues)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Reshape lagi ke (10000, 32, 32) karena kalau ga gini ga bakal terdisplay 
# (sebelumnya (10000,32,32,1))
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

fig, axes = plt.subplots(ncols=6, nrows=2, sharex=False,
    sharey=True, figsize=(17, 6))
index = 0
for i in range(2):
    for j in range(6):
        axes[i,j].set_title('actual:' + label_names[y_test[index][0]] + '\n' 
                            + 'predicted:' + label_names[predictions[index][0]])
        axes[i,j].imshow(X_test[index], cmap='gray')
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        index += 1
plt.show()